In [78]:
from Node import *
from Tokenizer import *
import random as rd
import pandas as pd

database_size = 1000000
file_name= 'dataset/ds1m.csv'

In [79]:
dataset = Node.create_dataset(2)

In [80]:
output_keys = []
output_info = []
for key in dataset:
    if dataset[key]['splits']:
        output_keys += [key]
        output_info += [dataset[key]['info']]

In [81]:
def add_split(sequence, allowed_split):
    split = rd.choice(allowed_split)
    if split in output_keys:
        new_split = rd.choices(dataset[split]['splits'],weights=dataset[split]['info_gain'])[0]
        if len(set(new_split) | set(sequence)) == 0:
            sequence.remove(split)
            sequence += new_split
    return sequence

In [82]:
labels = []
inputs = []

for label in rd.choices(output_keys, weights=output_info, k= database_size):
    sequence = rd.choices(dataset[label]['splits'],weights=dataset[label]['info_gain'])[0][:]
    allowed_split = [seq for seq in sequence if seq in output_keys]
    if allowed_split and rd.random() < 2/3:
        sequence = add_split(sequence, allowed_split)
        allowed_split = [seq for seq in sequence if seq in output_keys]

        if allowed_split and rd.random() < 1/2: sequence = add_split(sequence, allowed_split)
    
    rd.shuffle(sequence)
    inputs += [sequence]
    labels += [label]

In [83]:
tokenizer = Tokenizer(Node.all_formula_tokens)

In [84]:
def key_to_form(key, dataset):
    label = rd.choice(dataset[key]['formulas'])
    return rd.choice(label['notations'])

In [85]:
encoder_input = []
decoder_input = []

for i in range(len(labels)):
    decoder_input += [' '.join(tokenizer.tokenize(str(key_to_form(labels[i], dataset))))]
    decod = []
    for key in inputs[i]:
        decod += tokenizer.tokenize(str(key_to_form(key, dataset))) + [',']
    decod = decod[:-1]
    encoder_input += [' '.join(decod)]

In [86]:
database = pd.DataFrame()
database['encoder_input'] = encoder_input
database['decoder_input'] = decoder_input
database

encoder_input  \
0       ( ! p2 | p1 ) & ( ! p3 => ! p0 ) , ( ! p1 => !...   
1       p2 => ( ! p1 | ! p3 ) , ( ! p1 <=> p3 ) & ( p2...   
2       ( p2 & p3 ) & ( p0 => p1 ) , ( ! p3 <=> p0 ) |...   
3       ( p2 & ! p3 ) | ( ! p3 <=> p0 ) , p3 => ( ! p2...   
4       ( p2 <=> p3 ) => p1 , ( p2 <=> p3 ) => ( ! p0 ...   
...                                                   ...   
999995  p1 | ( ! p2 & p0 ) , ( p1 <=> p0 ) => ( ! p2 <...   
999996  ( ! p3 => ! p1 ) <=> ( p0 | p2 ) , ( p0 => ! p...   
999997  ( ! p2 & ! p1 ) => ( p0 => ! p3 ) , ( p2 => ! ...   
999998  ( p2 => ! p1 ) => ( p0 | p3 ) , ! p1 => ( p0 =...   
999999           ( ! p2 | p3 ) & p0 , p3 & ( ! p1 => p2 )   

                             decoder_input  
0         ( ! p2 & p0 ) & ( ! p1 => ! p3 )  
1                     p2 & ( ! p3 <=> p1 )  
2            ( p3 & ! p1 ) & ( p2 & ! p0 )  
3                     ( p2 | p3 ) <=> ! p0  
4                            ! p3 <=> ! p2  
...                                    ...  
999995    ( ! p2 => ! p1 ) <=> ( p2 | p0 )  
999996   ( p0 => ! p2 ) & ( ! p3 => ! p1 )  
999997  ( ! p2 & ! p1 ) <=> ( p0 => ! p3 )  
999998     ( ! p3 & ! p0 ) | ( p2 & ! p1 )  
999999        ( ! p2 => p1 ) & ( p0 & p3 )  

[1000000 rows x 2 columns]

In [87]:
database.to_csv(file_name)